# 🔐 Unsupervised Network Intrusion Detection

## Overview

This notebook implements an unsupervised machine learning approach to detect anomalous network traffic using K-Means clustering on the NSL-KDD dataset. The model is trained without using attack labels, learning traffic behavior patterns directly from the data. Ground-truth labels are used only for evaluation to assess how well the discovered clusters separate normal and malicious activity.

## Workflow

The notebook demonstrates a complete practical workflow including:

- Data preprocessing and feature encoding
- K-Means clustering for pattern discovery
- Performance analysis and evaluation metrics

## Dataset

The **NSL-KDD dataset** is an improved version of the KDD Cup 1999 dataset, containing network connection records with features describing connection characteristics and labeled attack types.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

####Load Data

In [4]:
import kagglehub

path = kagglehub.dataset_download("hassan06/nslkdd")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'nslkdd' dataset.
Path to dataset files: /kaggle/input/nslkdd


In [5]:
import os

os.listdir(path)

['KDDTest+.arff',
 'KDDTest-21.arff',
 'KDDTest1.jpg',
 'KDDTrain+.txt',
 'KDDTrain+_20Percent.txt',
 'KDDTest-21.txt',
 'KDDTest+.txt',
 'KDDTrain+.arff',
 'index.html',
 'nsl-kdd',
 'KDDTrain+_20Percent.arff',
 'KDDTrain1.jpg']

In [9]:
training_dataset = os.path.join(path, "KDDTrain+.txt")

train_df = pd.read_csv(training_dataset, header=None, sep=",")

In [10]:
display(train_df.head(10))

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,42
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal,20
1,0,udp,other,SF,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15
2,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,19
3,0,tcp,http,SF,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21
4,0,tcp,http,SF,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21
5,0,tcp,private,REJ,0,0,0,0,0,0,...,0.07,0.07,0.00,0.00,0.00,0.00,1.00,1.00,neptune,21
6,0,tcp,private,S0,0,0,0,0,0,0,...,0.04,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,21
7,0,tcp,private,S0,0,0,0,0,0,0,...,0.06,0.07,0.00,0.00,1.00,1.00,0.00,0.00,neptune,21
8,0,tcp,remote_job,S0,0,0,0,0,0,0,...,0.09,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,21
9,0,tcp,private,S0,0,0,0,0,0,0,...,0.05,0.06,0.00,0.00,1.00,1.00,0.00,0.00,neptune,21


In [11]:
print("Training Dataset shape: ", train_df.shape)
print("Training samples: ", train_df.shape[0])
print("Training features: ", train_df.shape[1])

Training Dataset shape:  (125973, 43)
Training samples:  125973
Training features:  43


In [12]:
testing_dataset = os.path.join(path, "KDDTest+.txt")

test_df = pd.read_csv(testing_dataset, header=None, sep=",")

In [13]:
display(test_df.head(10))

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,42
0,0,tcp,private,REJ,0,0,0,0,0,0,...,0.04,0.06,0.00,0.00,0.00,0.00,1.00,1.00,neptune,21
1,0,tcp,private,REJ,0,0,0,0,0,0,...,0.00,0.06,0.00,0.00,0.00,0.00,1.00,1.00,neptune,21
2,2,tcp,ftp_data,SF,12983,0,0,0,0,0,...,0.61,0.04,0.61,0.02,0.00,0.00,0.00,0.00,normal,21
3,0,icmp,eco_i,SF,20,0,0,0,0,0,...,1.00,0.00,1.00,0.28,0.00,0.00,0.00,0.00,saint,15
4,1,tcp,telnet,RSTO,0,15,0,0,0,0,...,0.31,0.17,0.03,0.02,0.00,0.00,0.83,0.71,mscan,11
5,0,tcp,http,SF,267,14515,0,0,0,0,...,1.00,0.00,0.01,0.03,0.01,0.00,0.00,0.00,normal,21
6,0,tcp,smtp,SF,1022,387,0,0,0,0,...,0.11,0.72,0.00,0.00,0.00,0.00,0.72,0.04,normal,21
7,0,tcp,telnet,SF,129,174,0,0,0,0,...,1.00,0.00,0.00,0.00,0.01,0.01,0.02,0.02,guess_passwd,15
8,0,tcp,http,SF,327,467,0,0,0,0,...,1.00,0.00,0.01,0.03,0.00,0.00,0.00,0.00,normal,21
9,0,tcp,ftp,SF,26,157,0,0,0,0,...,0.50,0.08,0.02,0.00,0.00,0.00,0.00,0.00,guess_passwd,7


In [14]:
print("Testing Dataset shape: ", test_df.shape)
print("Testing samples: ", test_df.shape[0])
print("Testing features: ", test_df.shape[1])

Testing Dataset shape:  (22544, 43)
Testing samples:  22544
Testing features:  43


####Assign Column Names

In [15]:
for df, name in [(train_df, "Train"), (test_df, "Test")]:
  df.columns = [f"features_{i}" for i in range(43)]
  print(f"{name} dataset: {df.shape[1]} columns named")

Train dataset: 43 columns named
Test dataset: 43 columns named


In [16]:
print("Re-Named Training Dataset:\n")

display(train_df.head(5))

Re-Named Training Dataset:



,features_0,features_1,features_2,features_3,features_4,features_5,features_6,features_7,features_8,features_9,...,features_33,features_34,features_35,features_36,features_37,features_38,features_39,features_40,features_41,features_42
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal,20
1,0,udp,other,SF,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15
2,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,19
3,0,tcp,http,SF,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21
4,0,tcp,http,SF,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21


In [17]:
print("Re-Named Testing Dataset:\n")

display(test_df.head(5))

Re-Named Testing Dataset:



,features_0,features_1,features_2,features_3,features_4,features_5,features_6,features_7,features_8,features_9,...,features_33,features_34,features_35,features_36,features_37,features_38,features_39,features_40,features_41,features_42
0,0,tcp,private,REJ,0,0,0,0,0,0,...,0.04,0.06,0.00,0.00,0.0,0.0,1.00,1.00,neptune,21
1,0,tcp,private,REJ,0,0,0,0,0,0,...,0.00,0.06,0.00,0.00,0.0,0.0,1.00,1.00,neptune,21
2,2,tcp,ftp_data,SF,12983,0,0,0,0,0,...,0.61,0.04,0.61,0.02,0.0,0.0,0.00,0.00,normal,21
3,0,icmp,eco_i,SF,20,0,0,0,0,0,...,1.00,0.00,1.00,0.28,0.0,0.0,0.00,0.00,saint,15
4,1,tcp,telnet,RSTO,0,15,0,0,0,0,...,0.31,0.17,0.03,0.02,0.0,0.0,0.83,0.71,mscan,11


####Separate Features and Labels

In [20]:
def prepare_features_labels(df):
  X = df.iloc[:, :-2]
  y = df.iloc[:, -2]
  difficulty = df.iloc[:, -1]
  return X, y

In [21]:
X_train, y_train = prepare_features_labels(train_df)
X_test, y_test = prepare_features_labels(test_df)

In [22]:
print("Training Features:", X_train.shape)
print("Training Labels:", y_train.shape)
print("Test features:", X_test.shape)
print("Test labels:", y_test.shape)

Training Features: (125973, 41)
Training Labels: (125973,)
Test features: (22544, 41)
Test labels: (22544,)


####Create Binary Attack labels

In [23]:
def create_binary_attack_labels(y):
  return (y != "normal").astype(int)

In [24]:
y_train_binary = create_binary_attack_labels(y_train)
y_test_binary = create_binary_attack_labels(y_test)

In [26]:
print(f"Training set: {np.sum(y_train_binary==0):,} normal, {np.sum(y_train_binary==1):,} neptune attacks")
print(f"Test set: {np.sum(y_test_binary==0):,} normal, {np.sum(y_test_binary==1):,} neptune attacks")

Training set: 67,343 normal, 58,630 neptune attacks
Test set: 9,711 normal, 12,833 neptune attacks


**normal** represents legitimate network traffic, while **neptune** is a Denial-of-Service (DoS) attack that floods a server with TCP SYN requests to exhaust system resources.

####Encode Categorical Features

In [29]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

def encode_categorical_features(X_train, X_test):

  cat_cols = X_train.select_dtypes(include='object').columns
  num_cols = X_train.select_dtypes(exclude='object').columns

  print(f"   Found {len(cat_cols)} categorical columns: {cat_cols.tolist()}")
  print(f"   Found {len(num_cols)} numerical columns \n")

  preprocessor = ColumnTransformer(
      transformers=[
          ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), cat_cols),
          ('num', 'passthrough', num_cols)
      ]
  )

  print("Fitting preprocessor on training data...\n")
  X_train_encoded = preprocessor.fit_transform(X_train)
  X_test_encoded = preprocessor.transform(X_test)

  feature_names = []
  feature_names.extend(num_cols)

  cat_encoder = preprocessor.named_transformers_['cat']
  cat_feature_names = cat_encoder.get_feature_names_out(cat_cols)
  feature_names.extend(cat_feature_names)

  X_train_encoded_df = pd.DataFrame(X_train_encoded, columns=feature_names, index=X_train.index)
  X_test_encoded_df = pd.DataFrame(X_test_encoded, columns=feature_names, index=X_test.index)

  print(f"   After encoding: {X_train_encoded_df.shape[1]} total features")
  print(f"   Training shape: {X_train_encoded_df.shape}")
  print(f"   Test shape:     {X_test_encoded_df.shape}")

  return X_train_encoded_df, X_test_encoded_df, preprocessor

In [30]:
X_train_encoded, X_test_encoded, preprocessor = encode_categorical_features(X_train, X_test)

   Found 3 categorical columns: ['features_1', 'features_2', 'features_3']
   Found 38 numerical columns 

Fitting preprocessor on training data...

   After encoding: 122 total features
   Training shape: (125973, 122)
   Test shape:     (22544, 122)
